In [4]:
import pandas as pd
import numpy as np
import os
import duckdb 

In [5]:
""" 
1. GUO cinesi e tutte le loro subsidiaries con codice NACE a 4 digits, paese e città 
(e, se c'è una divisione sub nazionale ma è importante solo per le subs cinesi). 
Oltre a questo ci serve GUO-name e SUBS- name e infine codice Nace del GUO.
2. Subsidiaries cinesi con info su città, NACE a 4 digits e paese del GUO."""

" \n1. GUO cinesi e tutte le loro subsidiaries con codice NACE a 4 digits, paese e città \n(e, se c'è una divisione sub nazionale ma è importante solo per le subs cinesi). \nOltre a questo ci serve GUO-name e SUBS- name e infine codice Nace del GUO.\n2. Subsidiaries cinesi con info su città, NACE a 4 digits e paese del GUO."

In [6]:
historicals_path = "..\\..\\data_raw\\ownership_history\\links_2016\\part-00000-0a8d598b-34d5-46fe-8e4e-12a02206fbc1-c000.snappy.parquet" 
key_financials_path = "..\\..\\data_raw\\key_financials\\key_financials_eur\part-00001-d5e63738-ec0f-45f1-9e6e-e90c27a22308-c000.snappy.parquet"
legal_info_path = "..\..\\data_raw\\firmographics\\legal_info\\part-00000-7ace73a0-3def-48c9-9b7c-99284c0e0049-c000.snappy.parquet"
contact_info = "..\..\\data_raw\\firmographics\\contact_info\\part-00074-54d921cc-a1c4-4ba5-941f-d71e06e76b2d-c000.snappy.parquet" 
# key_financials_detail_path = "..\\data_raw\key_financials_detailed\\key_financials\part-00001-00c555a2-204d-49c4-a64e-734e717b6eed-c000.snappy.parquet"

<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:4: SyntaxWarning: invalid escape sequence '\.'
<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:4: SyntaxWarning: invalid escape sequence '\.'
C:\Users\TR049283\AppData\Local\Temp\ipykernel_17852\76573579.py:2: SyntaxWarning: invalid escape sequence '\p'
  key_financials_path = "..\\..\\data_raw\\key_financials\\key_financials_eur\part-00001-d5e63738-ec0f-45f1-9e6e-e90c27a22308-c000.snappy.parquet"
C:\Users\TR049283\AppData\Local\Temp\ipykernel_17852\76573579.py:3: SyntaxWarning: invalid escape sequence '\.'
  legal_info_path = "..\..\\data_raw\\firmographics\\legal_info\\part-00000-7ace73a0-3def-48c9-9b7c-99284c0e0049-c000.snappy.parquet"
C:\Users\TR049283\AppData\Local\Temp\ipykernel_17852\76573579.py:4: SyntaxWarning: invalid escape sequence '\.'
  contact_info = "..\..\\data_raw\\firmographics\\contact_info\\part-00074-54d921cc

In [23]:
TEMP_TABLE_FIRMOGRAPHICS = "..\\..\\data_processed\\firmographics_processed"
TEMP_TABLE_KEY_FINANCIALS_DETAILED = "..\\..\\data_processed\\key_financials_detailed_processed"

def get_ownership_data(year, country, path=None):
    print(f"{country} - {year}...")

    COUNTRY_FIRMOGRAPHICS = TEMP_TABLE_FIRMOGRAPHICS + f"\\firmographics_{country}.parquet"
    COUNTRY_KEY_FINANCIALS = TEMP_TABLE_KEY_FINANCIALS_DETAILED + f"\\key_financials_detailed_{country}.parquet"

    query = f"""
        SELECT
            main.subsidiary_bvd_id,
            main.guo_25,
            {year} AS year,

            firmographics_sub.nuts2 AS subsidiary_nuts2,
            firmographics_sub.nace_rev_2_core_code_4_digits_ AS subsidiary_nace4,
            firmographics_guo.nuts2 AS guo_nuts2,
            firmographics_guo.nace_rev_2_core_code_4_digits_ AS guo_nace4,
            firmographics_guo.type_of_entity AS guo_type_of_entity,
            firmographics_guo.status AS guo_status,
            
            key_financials_detailed.number_of_employees
        FROM 
            '{path}' AS main

        LEFT JOIN 
            '{COUNTRY_KEY_FINANCIALS}' AS key_financials_detailed
        ON 
            main.guo_25 = key_financials_detailed.bvd_id_number
        AND 
            key_financials_detailed.year = {year}

        LEFT JOIN 
            '{COUNTRY_FIRMOGRAPHICS}' AS firmographics_sub
        ON 
            main.subsidiary_bvd_id = firmographics_sub.bvd_id_number

        LEFT JOIN 
            '{COUNTRY_FIRMOGRAPHICS}' AS firmographics_guo
        ON 
            main.guo_25 = firmographics_guo.bvd_id_number

        WHERE 
            main."type_of_relation" = 'GUO 25'
        AND 
            main."guo_25" LIKE '{country}%'
    """

    conn = duckdb.connect()
    df = conn.execute(query).fetchdf()
    return df


In [24]:
country = "CH"
year = "2012"
path = "..\\..\\data_raw\\ownership_history\\links_2012\\*.parquet"

df = get_ownership_data(
    path=path,
    year=year,
    country=country,
)

CH - 2012...


IOException: IO Error: No files found that match the pattern "..\..\data_processed\key_financials_detailed_processed\key_financials_detailed_CH.parquet"

In [8]:
TEMP_TABLE_FIRMOGRAPHICS = "..\\data_processed\\firmographics_processed"
TEMP_TABLE_KEY_FINANCIALS_DETAILED = "..\\data_processed\\key_financials_detailed_processed"
TEMP_TABLE_KEY_FINANCIALS = "..\\data_processed\\key_financials_processed"

path = "..\\data_raw\\ownership_history\\links_2017\\*.parquet"
year = 2012 
country = "CH"

COUNTRY_FIRMOGRAPHICS = TEMP_TABLE_FIRMOGRAPHICS + f"\\firmographics_{country}.parquet"
COUNTRY_KEY_FINANCIALS_DETAILED = TEMP_TABLE_KEY_FINANCIALS_DETAILED + f"\\key_financials_detailed_{country}.parquet"
COUNTRY_KEY_FINANCIALS = TEMP_TABLE_KEY_FINANCIALS + f"\\key_financials_{country}.parquet"


query_detailed = f"""
    SELECT
        main.subsidiary_bvd_id,
        main.guo_25,
        {year} AS year,

        firmographics_sub.nuts2 AS subsidiary_nuts2,
        firmographics_sub.nace_rev_2_core_code_4_digits_ AS subsidiary_nace4,
         
        key_financials_detailed.number_of_employees
    FROM 
        '{path}' AS main

    LEFT JOIN 
            '{COUNTRY_KEY_FINANCIALS_DETAILED}' AS key_financials_detailed
        ON 
            main.guo_25 = key_financials_detailed.bvd_id_number
        AND 
            key_financials_detailed.year = {year}

        LEFT JOIN 
            '{COUNTRY_FIRMOGRAPHICS}' AS firmographics_sub
        ON 
            main.subsidiary_bvd_id = firmographics_sub.bvd_id_number

    WHERE 
        main."type_of_relation" = 'GUO 25'
    AND 
        main."subsidiary_bvd_id" LIKE '{country}%'
"""

# conn = duckdb.connect()
# df = conn.execute(query_detailed).fetchdf()


In [9]:
TEMP_TABLE_FIRMOGRAPHICS = "..\\data_processed\\firmographics_processed"
TEMP_TABLE_KEY_FINANCIALS_DETAILED = "..\\data_processed\\key_financials_detailed_processed"
TEMP_TABLE_KEY_FINANCIALS = "..\\data_processed\\key_financials_processed"

path = "..\\data_raw\\ownership_history\\links_2017\\*.parquet"
year = 2012
country = "CH"

COUNTRY_FIRMOGRAPHICS = TEMP_TABLE_FIRMOGRAPHICS + f"\\firmographics_{country}.parquet"
COUNTRY_KEY_FINANCIALS_DETAILED = TEMP_TABLE_KEY_FINANCIALS_DETAILED + f"\\key_financials_detailed_{country}.parquet"
COUNTRY_KEY_FINANCIALS = TEMP_TABLE_KEY_FINANCIALS + f"\\key_financials_{country}.parquet"


query_detailed = f"""
    SELECT
        main.subsidiary_bvd_id,
        main.guo_25,
        {year} AS year,

        firmographics_sub.nuts2 AS subsidiary_nuts2,
        firmographics_sub.nace_rev_2_core_code_4_digits_ AS subsidiary_nace4,
        
        key_financials_detailed.number_of_employees,
        CASE 
            WHEN key_financials_detailed.bvd_id_number IS NOT NULL THEN 'key_financials_detailed'
            ELSE 'none'
        END AS source_key_financials,
        CASE 
            WHEN firmographics_sub.bvd_id_number IS NOT NULL THEN 'firmographics_sub'
            ELSE 'none'
        END AS source_firmographics
    FROM 
        '{path}' AS main

    LEFT JOIN 
        '{COUNTRY_KEY_FINANCIALS_DETAILED}' AS key_financials_detailed
        ON main.guo_25 = key_financials_detailed.bvd_id_number
        AND key_financials_detailed.year = {year}

    LEFT JOIN 
        '{COUNTRY_FIRMOGRAPHICS}' AS firmographics_sub
        ON main.subsidiary_bvd_id = firmographics_sub.bvd_id_number

    WHERE 
        main."type_of_relation" = 'GUO 25'
    AND 
        main."subsidiary_bvd_id" LIKE '{country}%'
"""

conn = duckdb.connect()
df = conn.execute(query_detailed).fetchdf()

IOException: IO Error: No files found that match the pattern "..\data_raw\ownership_history\links_2017\*.parquet"

## FIRMOGRAPHICS

In [5]:
def get_firmographics_data():
    query = f"""
        SELECT * 
        FROM '{TEMP_TABLE_FIRMOGRAPHICS}'
    """

    conn = duckdb.connect()
    df = conn.execute(query).fetchdf()
    return df



In [ ]:
df = pd.read_parquet(COUNTRY_FIRMOGRAPHICS)

In [43]:
italy_firmographics = pd.read_parquet(COUNTRY_FIRMOGRAPHICS)
italy_firmographics.to_csv("italy_firmographics.csv", index=False)

In [44]:
italy_key_financials_detailed = pd.read_parquet(COUNTRY_KEY_FINANCIALS_DETAILED)
italy_key_financials_detailed.to_csv("italy_key_financials_detailed.csv", index=False)

In [35]:
query_non_detailed = f"""
    SELECT
        main.subsidiary_bvd_id,
        main.guo_25,
        {year} AS year,

        firmographics_sub.nuts2 AS subsidiary_nuts2,
        firmographics_sub.nace_rev_2_core_code_4_digits_ AS subsidiary_nace4,
         
        key_financials_detailed.number_of_employees
    FROM 
        '{path}' AS main

    LEFT JOIN 
            '{COUNTRY_KEY_FINANCIALS}' AS key_financials
        ON 
            main.guo_25 = key_financials.bvd_id_number
        AND 
            key_financials.year = {year}

        LEFT JOIN 
            '{COUNTRY_FIRMOGRAPHICS}' AS firmographics_sub
        ON 
            main.subsidiary_bvd_id = firmographics_sub.bvd_id_number

    WHERE 
        main."type_of_relation" = 'GUO 25'
    AND 
        main."subsidiary_bvd_id" LIKE '{country}%'
"""

conn = duckdb.connect()
df2 = conn.execute(query_detailed).fetchdf()

In [30]:
# sort by bvd
df = df.sort_values(by = "subsidiary_bvd_id")
df

,subsidiary_bvd_id,guo_25,year,subsidiary_nuts2,subsidiary_nace4
8577368,IT*1040344617,IT*1040344617,2017,None,None
8577369,IT*1040455880,IT*1040455880,2017,None,None
8577370,IT*1040455881,IT*1040455881,2017,None,None
8577371,IT*1040506137,IT*1040506137,2017,None,None
8577372,IT*1040542490,IT*1040542490,2017,None,None
...,...,...,...,...,...
7005154,ITZZZZZA67S05Z330U,ITPPZZZZZA67S05Z330U,2017,ITC1 - Piemonte,None
7001320,ITZZZZZA67S05Z330U,ITPPZZZZZA67S05Z330U,2017,ITC1 - Piemonte,4941
7005100,ITZZZZZA67S05Z330U,ITPPZZZZZA67S05Z330U,2017,ITC1 - Piemonte,None
9767419,ITZZZZZA67S05Z330U,ITPPZZZZZA67S05Z330U,2017,ITC1 - Piemonte,None


In [31]:
# count diplicated in subsidiary_bvd_id
df.subsidiary_bvd_id.duplicated().sum()


np.int64(6083924)

In [17]:
path1 = "..\\data_processed\\guo_subs_europee\\guos_eu\\AT_2007_chunk_1.csv"
path2 = "..\\data_processed\\guo_subs_europee\\guos_eu\\AT_2007_chunk_2.csv"
path3 = "..\\data_processed\\guo_subs_europee\\guos_eu\\AT_2007_chunk_3.csv"
path4 = "..\\data_processed\\guo_subs_europee\\guos_eu\\AT_2007_chunk_4.csv"
path5 = "..\\data_processed\\guo_subs_europee\\guos_eu\\AT_2007_chunk_5.csv"

de1 = pd.read_csv(path1)
de2 = pd.read_csv(path2)
de3 = pd.read_csv(path3)
de4 = pd.read_csv(path4)
de5 = pd.read_csv(path5)

de = pd.concat([de1, de2, de3, de4, de5])
de

C:\Users\TR049283\AppData\Local\Temp\ipykernel_21044\1636585391.py:11: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  de5 = pd.read_csv(path5)


,subsidiary_bvd_id,guo_25,subsidiary_nuts2,subsidiary_nace4,guo_nuts2,guo_nace4,guo_type_of_entity,guo_status,year,guo_number_of_employees,guo_closing_date
0,AT9110479845,AT9110277823,AT13 - Wien,6832.0,AT13 - Wien,8299.0,Foundation/Research institute,Active,2007,NaN,NaN
1,DE8250295007,AT9110281795,DE21 - Oberbayern,4110.0,AT13 - Wien,NaN,Foundation/Research institute,Active,2007,NaN,NaN
2,AT9110387942,AT9110281788,AT13 - Wien,8299.0,AT13 - Wien,8299.0,Foundation/Research institute,Active,2007,NaN,NaN
3,AT9130020791,AT9110281787,AT32 - Salzburg,NaN,AT13 - Wien,8299.0,Foundation/Research institute,Active,2007,NaN,NaN
4,AT9110124882,AT9110281781,AT13 - Wien,7010.0,AT13 - Wien,8299.0,Foundation/Research institute,Active,2007,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
102837,CHCHE105633087,AT*Q0004169367,NaN,NaN,NaN,NaN,NaN,NaN,2007,NaN,NaN
102838,CHCHE105633087,AT*Q0004169367,NaN,NaN,NaN,NaN,NaN,NaN,2007,NaN,NaN
102839,CHCHE109610292,AT*Q000704580,NaN,NaN,NaN,NaN,NaN,NaN,2007,NaN,NaN
102840,GB01855172,AT*1525755171,NaN,NaN,NaN,NaN,NaN,NaN,2007,NaN,NaN


In [18]:
# check duplicates
print(de.shape)

de_ = de.drop_duplicates()

print(de_.shape)

(514210, 11)
(291100, 11)


In [19]:
path1 = "..\\data_processed\\guo_subs_europee\\guos_eu\\IT_2007_chunk_1.csv"
path2 = "..\\data_processed\\guo_subs_europee\\guos_eu\\IT_2007_chunk_2.csv"
path3 = "..\\data_processed\\guo_subs_europee\\guos_eu\\IT_2007_chunk_3.csv"
path4 = "..\\data_processed\\guo_subs_europee\\guos_eu\\IT_2007_chunk_4.csv"
path5 = "..\\data_processed\\guo_subs_europee\\guos_eu\\IT_2007_chunk_5.csv"

it1 = pd.read_csv(path1)
it2 = pd.read_csv(path2)
it3 = pd.read_csv(path3)
it4 = pd.read_csv(path4)
it5 = pd.read_csv(path5)

it = pd.concat([it1, it2, it3, it4, it5])
it

C:\Users\TR049283\AppData\Local\Temp\ipykernel_21044\651415005.py:10: DtypeWarning: Columns (4,6,7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  it4 = pd.read_csv(path4)
C:\Users\TR049283\AppData\Local\Temp\ipykernel_21044\651415005.py:11: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  it5 = pd.read_csv(path5)


,subsidiary_bvd_id,guo_25,subsidiary_nuts2,subsidiary_nace4,guo_nuts2,guo_nace4,guo_type_of_entity,guo_status,year,guo_number_of_employees,guo_closing_date
0,IT03934910260,IT03934910260,ITH3 - Veneto,7010.0,ITH3 - Veneto,NaN,Financial company,Dissolved (merger or take-over),2007,NaN,2007-12-31
1,IT03934910260,IT03934910260,ITH3 - Veneto,NaN,ITH3 - Veneto,NaN,Financial company,Dissolved (merger or take-over),2007,NaN,2007-12-31
2,IT00200000263,IT03934910260,ITH3 - Veneto,2752.0,ITH3 - Veneto,NaN,Financial company,Dissolved (merger or take-over),2007,NaN,2007-12-31
3,IT03934910260,IT03934910260,ITH3 - Veneto,7010.0,ITH3 - Veneto,7010.0,Financial company,Dissolved (merger or take-over),2007,NaN,2007-12-31
4,IT03934910260,IT03934910260,ITH3 - Veneto,NaN,ITH3 - Veneto,7010.0,Financial company,Dissolved (merger or take-over),2007,NaN,2007-12-31
...,...,...,...,...,...,...,...,...,...,...,...
93401,ESB97603708,IT04230130280,ES51 - Cataluna,4777.0,NaN,NaN,NaN,NaN,2007,NaN,2007-12-31
93402,IT04230130280,IT04230130280,NaN,NaN,NaN,NaN,NaN,NaN,2007,NaN,2007-12-31
93403,IT03577970969,IT04471520967,ITC4 - Lombardia,NaN,NaN,NaN,NaN,NaN,2007,NaN,2007-12-31
93404,IT03577970969,IT04471520967,ITC4 - Lombardia,7022.0,NaN,NaN,NaN,NaN,2007,NaN,2007-12-31


In [28]:
import duckdb

query = f"""
SELECT *
FROM parquet_scan('{key_financials_detail_path}')
LIMIT 10
"""

conn = duckdb.connect()
df = conn.execute(query).fetchdf()

print(df)


  bvd_id_number consolidation_code            filing_type closing_date  \
0  CN9385779231                 LF  Local registry filing   2020-12-31   
1  CN9385779233                 LF  Local registry filing   2018-12-31   
2  CN9385779234                 LF  Local registry filing   2017-12-31   
3  CN9385779235                 LF  Local registry filing   2021-12-31   
4  CN9385779236                 U1  Local registry filing   2018-12-31   
5  CN9385779236                 U1  Local registry filing   2017-12-31   
6  CN9385779236                 U1  Local registry filing   2016-12-31   
7  CN9385779236                 U1  Local registry filing   2014-12-31   
8  CN9385779237                 LF  Local registry filing   2020-12-31   
9  CN9385779238                 LF  Local registry filing   2018-12-31   

   number_of_months audit_status accounting_practice  \
0                12         None                None   
1                12         None                None   
2                

In [23]:
df = pd.read_parquet(key_financials_detail_path)
df.columns

Index(['bvd_id_number', 'consolidation_code', 'filing_type', 'closing_date',
       'number_of_months', 'audit_status', 'accounting_practice',
       'source_for_publicly_quoted_companies_', 'original_units',
       'original_currency', 'exchange_rate_from_original_currency',
       'fixed_assets', 'intangible_fixed_assets', 'tangible_fixed_assets',
       'other_fixed_assets', 'current_assets', 'stock', 'debtors',
       'other_current_assets', 'cash_cash_equivalent', 'total_assets',
       'shareholders_funds', 'capital', 'other_shareholders_funds',
       'non_current_liabilities', 'long_term_debt',
       'other_non_current_liabilities', 'provisions', 'current_liabilities',
       'loans', 'creditors', 'other_current_liabilities',
       'total_shareh_funds_liab_', 'working_capital', 'net_current_assets',
       'enterprise_value', 'number_of_employees',
       'operating_revenue_turnover_', 'sales', 'costs_of_goods_sold',
       'gross_profit', 'other_operating_expenses', 'operati

In [26]:
df.columns

Index(['bvd_id_number', 'consolidation_code', 'filing_type', 'closing_date',
       'number_of_months', 'audit_status', 'accounting_practice',
       'source_for_publicly_quoted_companies_', 'original_units',
       'original_currency', 'exchange_rate_from_original_currency',
       'fixed_assets', 'intangible_fixed_assets', 'tangible_fixed_assets',
       'other_fixed_assets', 'current_assets', 'stock', 'debtors',
       'other_current_assets', 'cash_cash_equivalent', 'total_assets',
       'shareholders_funds', 'capital', 'other_shareholders_funds',
       'non_current_liabilities', 'long_term_debt',
       'other_non_current_liabilities', 'provisions', 'current_liabilities',
       'loans', 'creditors', 'other_current_liabilities',
       'total_shareh_funds_liab_', 'working_capital', 'net_current_assets',
       'enterprise_value', 'number_of_employees',
       'operating_revenue_turnover_', 'sales', 'costs_of_goods_sold',
       'gross_profit', 'other_operating_expenses', 'operati

In [25]:
for col in df.columns:
    print(col)


bvd_id_number
consolidation_code
filing_type
closing_date
number_of_months
audit_status
accounting_practice
source_for_publicly_quoted_companies_
original_units
original_currency
exchange_rate_from_original_currency
fixed_assets
intangible_fixed_assets
tangible_fixed_assets
other_fixed_assets
current_assets
stock
debtors
other_current_assets
cash_cash_equivalent
total_assets
shareholders_funds
capital
other_shareholders_funds
non_current_liabilities
long_term_debt
other_non_current_liabilities
provisions
current_liabilities
loans
creditors
other_current_liabilities
total_shareh_funds_liab_
working_capital
net_current_assets
enterprise_value
number_of_employees
operating_revenue_turnover_
sales
costs_of_goods_sold
gross_profit
other_operating_expenses
operating_p_l_ebit_
financial_revenue
financial_expenses
financial_p_l
p_l_before_tax
taxation
p_l_after_tax
extr_and_other_revenue
extr_and_other_expenses
extr_and_other_p_l
p_l_for_period_net_income_
export_revenue
material_costs
costs_o

In [12]:
cols = ['bvd_id_number', 'operating_revenue_turnover_', 'number_of_employees', 'costs_of_goods_sold', 'closing_date']
df[cols].head()

,bvd_id_number,operating_revenue_turnover_,closing_date
0,CN9450018385,619000.0,2020-12-31
1,CN9450018386,605000.0,2019-12-31
2,CN9450018387,NaN,2018-12-31
3,CN9450018388,NaN,2019-12-31
4,CN9450018391,NaN,2018-12-31


In [ ]:
df.loc[(df.type_of_relation=='ISH') | (df.type_of_relation=='GUO 25')]

,subsidiary_bvd_id,subsidiary_independence_indicator,shareholder_bvd_id,shareholder_independence_indicator,direct_,direct_only_figures_,total_,total_only_figures_,information_date,source,type_of_relation,active_archived,guo_25,guo_50,guo_50c,guo_25c,guo_25_jo,guo_25c_jo
0,RO15110869,B+,RO*110021000020,-,50.00,50.00,n.a.,None,20160809,VD,GUO 25,active,RO*110021000020,None,None,RO15110869,None,None
4,RO15110869,B+,RO*110021000020,-,50.00,50.00,n.a.,None,20160809,CC,ISH,active,RO*110021000020,None,None,RO15110869,None,None
6,RO15110893,D,RO*554672356,-,100.00,100.00,100.00,100.00,20160809,VD,GUO 25,active,RO*554672356,RO*554672356,RO15110893,RO15110893,None,None
14,RO15110893,D,RO*554672356,-,100.00,100.00,100.00,100.00,20160809,CC,ISH,active,RO*554672356,RO*554672356,RO15110893,RO15110893,None,None
17,RO15110923,D,RO*110143075806,-,80.00,80.00,n.a.,None,20160809,VD,GUO 25,active,RO*110143075806,RO*110143075806,RO15110923,RO15110923,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3148785,RO24439357,B+,RO*554033787,-,50.00,50.00,n.a.,None,20160423,VD,GUO 25,active,RO*554033786,None,None,RO24439357,RO*554033787,None
3148790,RO24439365,B+,RO*554033788,-,50.00,50.00,n.a.,None,20160814,VD,GUO 25,active,RO*554033788,None,None,RO24439365,RO*555702630,None
3148791,RO24439365,B+,RO*555702630,-,50.00,50.00,n.a.,None,20160814,VD,GUO 25,active,RO*554033788,None,None,RO24439365,RO*555702630,None
3148796,RO24439373,B+,RO*554033790,-,50.00,50.00,n.a.,None,20160809,VD,GUO 25,active,RO*554033790,None,None,RO24439373,RO*554033791,None


: 

In [ ]:
subdirs = [d for d in os.listdir('ownership_history') if 'links' in d and os.path.isdir(os.path.join('ownership_history', d))]

files_dict = {}

for subdir in subdirs:
    subdir_path = os.path.join('ownership_history', subdir)
    files_dict[subdir] = [os.path.join(subdir_path, f) for f in os.listdir(subdir_path)]

In [ ]:
path = files_dict['links_2007'][0]

In [ ]:
test = pd.read_parquet(path)

In [ ]:
test.to_csv('test.csv', index=False)

In [ ]:
test

In [ ]:
import dask.dataframe as dd

european_countries = ['GB', 'FR', 'DE', 'IT', 'ES', 'NL', 'BE', 'PL', 'SE', 'DK', 'FI', 'PT', 'GR', 'IE', 'AT', 'CH', 'NO']

# Extract the first two characters of 'bvd_id_number' and filter based on European countries


test['subsidiary_country_code'] = test['subsidiary_bvd_id'].str[:2]
test['guo_country_code'] = test['shareholder_bvd_id'].str[:2]

# Keep rows where either 'bvd_country_code' or 'shareholder_country_code' is in the list of European countries
industry_classifications = test[
    test['subsidiary_country_code'].isin(european_countries) | 
    test['guo_country_code'].isin(european_countries)
]

industry_classifications

In [ ]:
test['type_of_relation'].unique()
test = test[~test['type_of_relation'].str.contains('DUO')]

In [ ]:
test[~test['type_of_relation'].str.contains('DUO') & test['guo_25'].notnull()]['direct_only_figures_']

In [ ]:
files = [f for f in os.listdir('firmografics\\all_addresses') if f.endswith('.parquet')]
full_paths = [os.path.join('firmografics\\all_addresses', f) for f in files]

all_addresses = pd.read_parquet(full_paths[0])

In [ ]:
len(full_paths)

In [ ]:
len(all_addresses.bvd_id_number.unique())

In [ ]:
files = [f for f in os.listdir('firmografics\\contact_info') if f.endswith('.parquet')]
full_paths = [os.path.join('firmografics\\contact_info', f) for f in files]

# contact_info = pd.read_parquet(full_paths[0])

In [ ]:
files = [f for f in os.listdir('firmografics\\industry_classifications') if f.endswith('.parquet')]
full_paths = [os.path.join('firmografics\\industry_classifications', f) for f in files]

industry_classifications = pd.read_parquet(full_paths[0])

_________________

In [ ]:
from pathlib import Path


In [ ]:
output_dir = Path('stata_files\\firmografics\\all_addresses')
output_dir.mkdir(parents=True, exist_ok=True)
input_dir = 'firmografics\\all_addresses'

files = [f for f in os.listdir(input_dir) if f.endswith('.parquet')]
full_paths = [os.path.join(input_dir, f) for f in files]


In [ ]:
for idx, file in enumerate(full_paths):
    df = pd.read_parquet(file)
    df = df[['bvd_id_number', 'postcode', 'country_iso_code']]
    df.to_csv(f'stata_files\\firmografics\\all_addresses\\chunk_{idx}.csv', index=False)


In [ ]:
output_dir = Path('stata_files\\firmografics\\contact_info')
output_dir.mkdir(parents=True, exist_ok=True)
input_dir = 'firmografics\\contact_info'

files = [f for f in os.listdir(input_dir) if f.endswith('.parquet')]
full_paths = [os.path.join(input_dir, f) for f in files]


In [ ]:
def dms_to_decimal(dms):
    import re
    try:
        pattern = re.compile(r'(\d+)°\s*(\d+)\'\s*([\d.]+)"\s*([NSEW])')
        match = pattern.match(dms)
        
        if match:
            degrees, minutes, seconds, direction = match.groups()
            decimal = float(degrees) + float(minutes)/60 + float(seconds)/3600
            
            if direction in ['S', 'W']:
                decimal = -decimal
            
            return decimal
        else:
            return np.nan 
    except Exception:
        return np.nan  


In [ ]:
for file in full_paths:
    df = pd.read_parquet(file)
    variables = ['bvd_id_number', 'country_iso_code', 'latitude', 'longitude']
    df = df[variables]
    df['latitude'] = df['latitude'].apply(dms_to_decimal)
    df['longitude'] = df['longitude'].apply(dms_to_decimal)
    df.to_csv(f'stata_files\\firmografics\\contact_info\\chunk_{idx}.csv', index=False)

In [ ]:
file1['latitude'].apply(dms_to_decimal)
file1['longitude'].apply(dms_to_decimal)

In [ ]:
1file1['latitude'] = file1['latitude'].apply(dms_to_decimal)
file1['longitude'] = file1['longitude'].apply(dms_to_decimal)
file1
